## RSA and the Gronsfeld Cipher

<hr />

# Table of Contents

### 1. Introduction  

### 2. RSA Code Functions  

### 3. Code Demo

### 4. Message Demo

### 5. Code Breaking Overview     

### 6. Code breaking complete examples  

### 7. Frequency Analysis and Gronsfeld Cipher










### 1. Introduction

The project works through the basic tools of RSA encryption.  It uses fast modular exponentiation and euclid's algorithm to find public keys and the extended euclidean algorithm.  These functions taken togother provide the basis for encryption and decryption. The project then looks at the brute force method to attack RSA by factoring the public key which can break the encryption.  The project then goes on to discuss other weaknesses in RSA and attack by frequency anaylsis.  The project walks through the difficulty in scrambling letters by showing how the Gronsfeld cipher is also subject to frequenct analysis.

### 2 RSA Code Functions
#### Basic processing tools



In [1]:
def Convert_Text(_string):
    integer_list = []
    for i in _string:
        integer = ord(i) #convert current letter to ascii
        integer_list.append(integer) #add converted letter to list
    return integer_list 


In [2]:
def Convert_Num(_list):
    _string = ''
    for i in _list:
        _string += chr(i) #convert int to string 
    return _string


In [3]:
def Convert_Binary_String(_int):
    n = _int
    l = [] 
    if n == 0: ## cover case where n = 0, otherwise while loop will terminate
        l.append(0) ## add 0 to output 
    while n > 0: ##while continious mod 2 is >0
        k = n % 2 ## take mod of number
        l.append(k) ## append mod to list.  NOTE: this will create the binary number in REVERSE order
        ## but this is okay since we will read it in reverse order  in FME below
        n = n // 2 ## divide n by 2 in prep for the next mod
    
    return l



### Mathematical functions


In [4]:
def FME(a, n, b):
    result = 1 
    square = a 
    for i in n: ## this loop reads each digit in our reversed binary list
        k = i % 2 ## take the mod of each digit
        if k == 1: ##if the mod is one, we need to update our result
            result = (result*square)% b ##new result is the mod of the current square value
        square = (square * square)% b ##for each binary digit, variable is squared to keep with current power value
    return result

In [5]:
def Euclidean_Alg(a, b):
    k = 1
    if a < b: #swaps a and b if b > a
        c = a 
        a = b
        b = c
    while(k > 0): #loop will stop when we get mod value of 0
        k = a % b # mod of a, b
        a = b # next time through, take mod of smaller number
        b = k # next time through, divisor will be answer of a mod b
    return a

print(Euclidean_Alg(141, 19))


1



### Key Generating Functions


In [6]:
def Find_Public_Key_e(p, q):
    n = p*q #n component
    n1 = (p-1)*(q-1) #this is necessary to apply fermat's little theorem
    e = n//2 #add diversity in output of e values, otherwise program will return 3 most of the time
    while True: #continue until we hit a return 
        test = Euclidean_Alg(n1, e) # apply Euclidean Alg to find relative prime between e and n1
        if test == 1 and e != p and e != q: #if gcd is one, exit loop and return n,e
            return n, e
        e += 1
    return n,e
print(Find_Public_Key_e(31,71))

(2201, 1103)


In [7]:
def EEA (m,n):
        m0 = m #save initial m
        n0 = n #save initial n
        s1, t1 = 1, 0
        s2, t2, = 0, 1
        m = s1 * m0 + t1 * n0
        n = s2 * m0 + t2 * n0
        while n > 0: #loop will terminate when successive mods = 0
            k = m % n
            q1 = m // n #allows us to calulate next bezout coefficient
            m = n #next time around, take mod of n
            n = k #by divisor k (answer of previous mod)
            s1h, t1h, = s2, t2 #save current s2 t2 value
            s2h, t2h = s1 - (q1 * s2), t1 - (q1 * t2) #calculate new coefficients
            s1, t1 = s1h, t1h #update s1,t1 for next round
            s2, t2 = s2h, t2h #update st,t2 for next round
        return s1 #when mod = 0 s1 and t1 are our bezout coefficeients, we are only interested in s1

def Find_Private_Key_d(e, p, q):
        n = p*q #n key component
        n1 = (p-1)*(q-1) #enables use of Fermatt's 
        d = EEA(e,n1) #fermat's little theorem
        while d < 0: #cannot use negative number so this will add n1 until we get a posative value
            d+= n1
        return (n,d)
print(Find_Private_Key_d(1103, 31, 71))

(2201, 1367)


### Encode and Decode Functions


In [8]:
def Encode(n, e, message):
    m = Convert_Text(message) #Converts text into an integer list
    cipher_text = []
    for num in m: #iterate through our integer lest
        c = FME(num, Convert_Binary_String(e), n) # use FME to find (converted integer value)^e mod n. As defined, FME relies on e being converted to binary rep
                                                  # answer is an integer which represents our encrypted letter
        cipher_text.append(c) #populate list with our encrpted letters
    return cipher_text
print(Encode(2201, 1103, "I don't want to talk to you no more, you empty-headed animal food trough wiper. I fart in your general direction. Your mother was a hamster and your father smelt of elderberries...Now go away or I shall taunt you a second time."))

[12, 900, 10, 1154, 1656, 946, 1937, 900, 1308, 2110, 1656, 1937, 900, 1937, 1154, 900, 1937, 2110, 2104, 1568, 900, 1937, 1154, 900, 2190, 1154, 1912, 900, 1656, 1154, 900, 1957, 1154, 513, 884, 1078, 900, 2190, 1154, 1912, 900, 884, 1957, 536, 1937, 2190, 1227, 1593, 884, 2110, 10, 884, 10, 900, 2110, 1656, 239, 1957, 2110, 2104, 900, 834, 1154, 1154, 10, 900, 1937, 513, 1154, 1912, 758, 1593, 900, 1308, 239, 536, 884, 513, 492, 900, 12, 900, 834, 2110, 513, 1937, 900, 239, 1656, 900, 2190, 1154, 1912, 513, 900, 758, 884, 1656, 884, 513, 2110, 2104, 900, 10, 239, 513, 884, 1049, 1937, 239, 1154, 1656, 492, 900, 959, 1154, 1912, 513, 900, 1957, 1154, 1937, 1593, 884, 513, 900, 1308, 2110, 1646, 900, 2110, 900, 1593, 2110, 1957, 1646, 1937, 884, 513, 900, 2110, 1656, 10, 900, 2190, 1154, 1912, 513, 900, 834, 2110, 1937, 1593, 884, 513, 900, 1646, 1957, 884, 2104, 1937, 900, 1154, 834, 900, 884, 2104, 10, 884, 513, 1265, 884, 513, 513, 239, 884, 1646, 492, 492, 492, 1554, 1154, 1308, 90

In [9]:
def Decode(n, d, cipher_text):
    message = '' 
    message_list = []
    for num in cipher_text: #iterate through our cipher text
        m = FME(num, Convert_Binary_String(d), n) # Use FME to find (encrypted digity)^d mod n
        message_list.append(m) # Append answer to list
    message = Convert_Num(message_list) #convert integers back to a string
    return message
text =  [12, 900, 10, 1154, 1656, 946, 1937, 900, 1308, 2110, 1656, 1937, 900, 1937, 1154, 900, 1937, 2110, 2104, 1568, 900, 1937, 1154, 900, 2190, 1154, 1912, 900, 1656, 1154, 900, 1957, 1154, 513, 884, 1078, 900, 2190, 1154, 1912, 900, 884, 1957, 536, 1937, 2190, 1227, 1593, 884, 2110, 10, 884, 10, 900, 2110, 1656, 239, 1957, 2110, 2104, 900, 834, 1154, 1154, 10, 900, 1937, 513, 1154, 1912, 758, 1593, 900, 1308, 239, 536, 884, 513, 492, 900, 12, 900, 834, 2110, 513, 1937, 900, 239, 1656, 900, 2190, 1154, 1912, 513, 900, 758, 884, 1656, 884, 513, 2110, 2104, 900, 10, 239, 513, 884, 1049, 1937, 239, 1154, 1656, 492, 900, 959, 1154, 1912, 513, 900, 1957, 1154, 1937, 1593, 884, 513, 900, 1308, 2110, 1646, 900, 2110, 900, 1593, 2110, 1957, 1646, 1937, 884, 513, 900, 2110, 1656, 10, 900, 2190, 1154, 1912, 513, 900, 834, 2110, 1937, 1593, 884, 513, 900, 1646, 1957, 884, 2104, 1937, 900, 1154, 834, 900, 884, 2104, 10, 884, 513, 1265, 884, 513, 513, 239, 884, 1646, 492, 492, 492, 1554, 1154, 1308, 900, 758, 1154, 900, 2110, 1308, 2110, 2190, 900, 1154, 513, 900, 12, 900, 1646, 1593, 2110, 2104, 2104, 900, 1937, 2110, 1912, 1656, 1937, 900, 2190, 1154, 1912, 900, 2110, 900, 1646, 884, 1049, 1154, 1656, 10, 900, 1937, 239, 1957, 884, 492, 900, 1227, 1227, 1794, 1154, 1656, 1937, 2190, 900, 1991, 2190, 1937, 1593, 1154, 1656, 900, 2110, 1656, 10, 900, 1937, 1593, 884, 900, 1593, 1154, 2104, 2190, 900, 758, 513, 2110, 239, 2104]
print(Decode(2201, 1367,text))

I don't want to talk to you no more, you empty-headed animal food trough wiper. I fart in your general direction. Your mother was a hamster and your father smelt of elderberries...Now go away or I shall taunt you a second time. --Monty Python and the holy grail


### 3. Putting the Pieces Togother-- Code Demo

In [10]:
def Find_Public_Key_e(p, q):
    n = p*q #n component
    n1 = (p-1)*(q-1) #this is necessary to apply fermat's little theorem
    e = n//2 #add diversity in output of e values, otherwise program will return 3 most of the time
    while True: #continue until we hit a return 
        test = Euclidean_Alg(n1, e) # apply Euclidean Alg to find relative prime between e and n1
        if test == 1 and e != p and e != q: #if gcd is one, exit loop and return n,e
            return n, e       
        e += 1
    return n,e
print(Find_Public_Key_e(31,71))

(2201, 1103)


### Public Key
The function above will find find a public key (n, e) from two prime numbers which are mannually entered. e is calculated by finding a relative prime to (p-1)(q-1) using the Euclidiean Alg.  The function starts with e = n//2 to add variety in calulated e values for different p and q values.

In [11]:
def Find_Private_Key_d(e, p, q):
        n = p*q #n key component
        n1 = (p-1)*(q-1) #enables use of Fermatt's 
        d = EEA(e,n1) #fermat's little theorem
        while d < 0: #cannot use negative number so this will add n1 until we get a posative value
            d+= n1
        return (n,d)
    
print(Find_Private_Key_d(1103, 31, 71))

(2201, 1367)


### Private key

The code above will take the e value from our public key and find the private key. Values have been manually enetered using the same p and q as the public key.  To find the private key, we must find the inverse of e mod (p-q)(q-1).  This is done using the Extended Euclidean Algo (function definition not shown, function defined in section 2.3).  The EEA will return only the bezout coefficient we are interested in, which is the inverse of e mod (p-1)(q-1).  Note that we cannot use negative numbers as e,d are used as powers.  If either were negative, it would result in a non-integer value.  The loop adds (p-q)(q-1) to d until it is posative.

In [12]:
def Encode(n, e, message):
    m = Convert_Text(message) #Converts text into an integer list
    cipher_text = []
    for num in m: #iterate through our integer lest
        c = FME(num, Convert_Binary_String(e), n) # use FME to find (converted integer value)^e mod n. As defined, FME relies on e being converted to binary rep
                                                  # answer is an integer which represents our encrypted letter
        cipher_text.append(c) #populate list with our encrpted letters
    return cipher_text
print(Encode(2201, 1103, "Hello, today is a good day."))

[2060, 884, 2104, 2104, 1154, 1078, 900, 1937, 1154, 10, 2110, 2190, 900, 239, 1646, 900, 2110, 900, 758, 1154, 1154, 10, 900, 10, 2110, 2190, 492]


### Encode

This function takes in the previously created public key, (n, e) as well as a string of text that the user wished to encode.  The message is first converted to an integer list using standard alpha numeric subsitution.  This integer representations are iterated through and used as the base in b^e mod n.  This is calculated using the Euclidean Alg.  The resulting integer is thus 'encoded' and added to the output.  A sample encrption is provided using the same public key as before.

In [13]:
def Decode(n, d, cipher_text):
    message = '' 
    message_list = []
    for num in cipher_text: #iterate through our cipher text
        m = FME(num, Convert_Binary_String(d), n) # Use FME to find (encrypted digity)^d mod n
        message_list.append(m) # Append answer to list
    message = Convert_Num(message_list) #convert integers back to a string
    return message
text =  [2060, 884, 2104, 2104, 1154, 1078, 900, 1937, 1154, 10, 2110, 2190, 900, 239, 1646, 900, 2110, 900, 758, 1154, 1154, 10, 900, 10, 2110, 2190, 492]

print(Decode(2201, 1367,text))

Hello, today is a good day.


### Decode

The decode function takes in the encrypted integer list and iterates through it.  Since the encrypted digit (b) comes from b^e mod n, we can undo the encryption by applying the inverse, d where the decoded b will be b^d mod n.  In the end, we see that the public and private keys create the equation b^(ed) mod n where raising b to e creates an ecrypted digit while raising the resulting b^d where d is the inverse, undoes the ecryption.  The decoding demo uses the piazza test post private key and encoded message.

### 4. Message Demo



In [14]:
n,e = (57599,11)
n,d = (57599,20771)
message = [44672, 23084, 26260, 27280, 30976, 15835, 26260, 13364, 32397, 32397, 13364, 30976, 26260, 13364, 32397, 30976, 47716, 11371, 8571, 30976, 46216, 8571, 13364, 15982, 8571, 57475, 21876, 40387, 30976, 26260, 50778, 27280, 32397, 13364, 30976, 40387, 13364, 26260, 52351, 8571, 26260, 27280, 57475, 11371, 21876, 35579]
message2 = [27012, 13364, 11371, 50778, 32397, 15982, 15982, 57475, 11371, 21876, 26260, 55139, 30976, 47742, 13364, 32397, 15982, 27280, 55139, 32397, 13364]
print(Decode(n,d, message))
print(Decode(n,d, message2))

What career are you pursuing after graduation?
Professional wrestler


In [15]:
n,e = ( 552367 , 3 )
n,d = ( 552367 , 367227 )
message = [106136, 20130, 360306, 456162, 32768, 52891, 416141, 32768, 114460, 262897, 496879, 376810, 32768, 508841, 360306, 538298, 262897, 376810, 52891, 456162, 477934, 32768, 300194, 52891, 158747, 158747, 360306, 32768, 456162, 262897, 300194, 300194, 52891, 226266, 540360, 250047]
message2 =  [274625, 226266, 114460, 262897, 226266, 477934, 32768, 28058, 20130, 262897, 32768, 447633, 262897, 477934, 416141, 226266, 59319, 456162, 32768, 416141, 360306, 114460, 32768, 300194, 477934, 300194, 300194, 477934, 376810, 262897, 226266, 52891, 32768, 52891, 416141, 32768, 28058, 376810, 262897, 226266, 540360, 97336]
print(Decode(n,d, message))
print(Decode(n,d, message2))

What is your favorite pizza topping?
Anyone who doesn't say pepperoni is wrong.


In [16]:
n,e = (3589, 41)
n,d = (3589, 1433)
message = [1515, 2507, 970, 2977, 723, 2029, 2504, 723, 1173, 666, 1597, 1945, 723, 1074, 3341, 2977, 1512, 2504, 723, 887, 970, 1152, 3341, 3524, 723, 970, 887, 787, 723, 2835, 2507, 1173, 3229]
message2 =  [2988, 666, 363, 1492, 3341, 1945, 723, 2504, 1074, 970, 887, 2029, 3341, 682, 723, 887, 970, 1152, 3341, 787, 723, 2226, 2029, 1074, 3341, 1945, 2661, 723, 1263, 3341, 723, 2835, 970, 2504, 723, 970, 682, 1945, 3341, 970, 787, 1173, 723, 93, 723, 1152, 666, 887, 2977, 2507, 2504, 723, 666, 682, 787, 723, 2835, 2507, 3341, 887, 723, 961, 723, 1568, 666, 2977, 723, 2507, 2029, 1152, 3524, 723, 2504, 666, 723, 2507, 3341, 723, 2835, 970, 2504, 723, 970, 682, 1945, 3341, 970, 787, 1173, 723, 1597, 2504, 3341, 787, 723, 2977, 666, 723, 2029, 2977, 2661, 723, 2853, 2208]
print(Decode(n,d, message))
print(Decode(n,d, message2))

What is your pet's name, and why?
Cocker spaniel named Piper. He was already 6 months old when I got him, so he was already used to it. :)


### 5. Code Breaking


Brute force code breaking works because the encryption relies on the multiplied value of n = pq.  From this, we derive the public key and private key.  Given that n will be publicly availible, all we need to do is factor n to find p and q.  From there, we can recreate the private key.  The strength of RSA is that given large enough p and q, factoring n becomes extermely difficult.  

Brute Force Factoring
<pre><code>def factorize(n):
    # n is a number, return the smallest factor of n
    for i from 2 to n-1:
        if i divides n:
            return i
        return FALSE
        </code></pre>


In [17]:
def factorize(n):
    p = 2
    for _ in range(n-1):
        if n % p == 0:
            q = n // p
            return (p,q)
        p += 1
print(factorize(703))

(19, 37)


### 6. Code Breaking Complete Examples

In [18]:
public_key = [703, 113]
encoded_message = [544, 472, 233, 242, 250, 555, 591, 550, 376, 364, 210, 165, 526, 47, 555, 95, 476, 210, 242, 165, 95, 233, 526, 210, 376, 95, 233, 242, 364, 210, 242, 224, 376, 95, 364, 233, 47, 242, 210, 555, 376, 165, 472, 242, 555, 410, 242, 250, 95, 233, 233, 47, 145]\

n = public_key[0] #pull n value
p,q = factorize(n) #pull out prime factors from n
e = public_key[1] #pull e value
d = Find_Private_Key_d(e, p, q) #plug public key, p and q in to find d, prive key.
print(Decode(*d, encoded_message)) #decode using private key




The Conquistador's treasure is buried south of Creed.


In [19]:
public_key = [53743,7]

Cipher = [31003, 31003, 31003, 31003, 36768, 36768, 36768, 36768, 3802, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 3802, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 36768, 36768, 3802, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 36768, 36768, 3802, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 3802, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 36768, 36768, 36768, 31003, 31003, 31003, 31003, 36768, 36768, 3802, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 36768, 3802, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 3802, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 36768, 36768, 3802, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 3802, 31003, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 3802, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 43852, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 43852, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 3802, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 50191, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 3802, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 3802, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 37053, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 3802, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 31003, 36768, 36768, 3802]
n = public_key[0] #pull n value
p,q = factorize(n) #pull out prime factors from n
e = public_key[1] #pull e value
d = Find_Private_Key_d(e, p, q) #plug public key, p and q in to find d, prive key.
print(Decode(*d, Cipher)) #decode using private key



....▓▓▓▓
..▓▓......▓
..▓▓......▓▓..................▓▓▓▓
..▓▓......▓▓..............▓▓......▓▓▓▓
..▓▓....▓▓..............▓......▓▓......▓▓
....▓▓....▓............▓....▓▓....▓▓▓....▓▓
......▓▓....▓........▓....▓▓..........▓▓....▓
........▓▓..▓▓....▓▓..▓▓................▓▓
........▓▓......▓▓....▓▓
.......▓......................▓
.....▓.........................▓
....▓......^..........^......▓
....▓............❤............▓
....▓..........................▓
......▓..........ٮ..........▓
..........▓▓..........▓▓



In [20]:
public_key = (81337, 5)

Cipher = [35474, 43588, 77066, 64281, 9038, 58709, 43588, 7077, 58709, 77066, 77066, 64281, 48652, 22144, 43588, 22535, 23808, 22535, 43588, 41840, 76598, 9038, 58709, 70493, 46125, 43588, 64484, 10670, 7077, 43588, 64484, 58709, 59796, 23808, 10670, 70493, 58709, 43588, 35474, 43588, 22535, 76598, 48652, 21466, 7077, 43588, 59227, 23808, 915, 58709, 43588, 23808, 43588, 59796, 59227, 64281, 77066, 22535, 43588, 64281, 7077, 21466, 70493, 43588, 23808, 43588, 64484, 64281, 7077, 43588, 76598, 42315, 43588, 23808, 43588, 42315, 23808, 10670, 15601, 43588, 47705, 23808, 70493, 17692]
n = public_key[0] #pull n value
p,q = factorize(n) #pull out prime factors from n
e = public_key[1] #pull e value
d = Find_Private_Key_d(e, p, q) #plug public key, p and q in to find d, prive key.
print(Decode(*d, Cipher)) #decode using private key


I like telling dad jokes, but because I don't have a child it's a bit of a faux pas.


Below demonstrates a brute force attack on a large n.  When the code runs, it never finishes.  Using brute force, it takes python an impractiacl amount of time to factor n.  The attack begins to fail when n is > 12 digits.  The function call that results in a runtime error has been commented out so that the whole kernal can be run with out interruption.  

In [21]:
public_key =  [3424458732409217653815525644407522211465790678007727512565605759,11]
Cipher: [2161283703465490489863, 11156683466653165551101, 23316389970546096340992, 23316389970546096340992, 1951354384207722496, 1951354384207722496, 1951354384207722496, 36028797018963968, 488595558857835544576, 31517572945366073781711, 31517572945366073781711, 21048519522998348950643, 46523913960640966796875, 36028797018963968, 23316389970546096340992, 17103393581163134765625, 21048519522998348950643, 11156683466653165551101, 36028797018963968, 25804264053054077850709, 81402749386839761113321, 36028797018963968, 34785499933455142617088, 42262322980951656843264, 17103393581163134765625, 25804264053054077850709, 11156683466653165551101, 46523913960640966796875, 36028797018963968, 67766737102405685929319, 11156683466653165551101, 42262322980951656843264, 11156683466653165551101, 28531167061100000000000, 317475837322472439, 51172646912339021398016, 36028797018963968, 34785499933455142617088, 42262322980951656843264, 17103393581163134765625, 25804264053054077850709, 11156683466653165551101, 36028797018963968, 11156683466653165551101, 28531167061100000000000, 31517572945366073781711, 56239892154164025151533, 13842338707244455781047, 15394540563150776827904, 36028797018963968, 7153014030880804126753, 12433743083946522728448, 51172646912339021398016, 11156683466653165551101, 42262322980951656843264, 36028797018963968, 7153014030880804126753, 23316389970546096340992, 23316389970546096340992, 1951354384207722496, 1951354384207722496, 1951354384207722496]
n = public_key[0] #pull n value
#p,q = factorize(n) #pull out prime factors from n
e = public_key[1] #pull e value
d = Find_Private_Key_d(e, p, q) #plug public key, p and q in to find d, prive key.
#print(Decode(*d, Cipher)) 

#decode using private key





### 7. Frequency Analysis and the Gronsfeld Cipher ##


With our implementation and the tools to find and process large primes, it seems that we would be able to come up with an n that is not practically factorable.  However, factoring n is not necessarily the only way to crack an RSA code.  Without additional encryption methods, RSA substitutes one letter for one number.  The problem with this is that there are common arrangements in letters and words that make guessing the code possible without doing too much math.

For example, imagine a war pilot picks up some ecrypted signal as he is flying over the ocean:

[1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009, 1160, 1009]

Given the context (out in the ocean) and the pattern (repeating two number sequence), we can be pretty sure this is an SOS signal and so we've cracked two letters just using common sense.  This would still be true if the n used were much larger than in this example.  

One of the ways that RSA could be made more secure would be to use letter scrambling in addition RSA.  Prior to feeding letters into the RSA scheme, they could be offset to scramble the letters.  Ancient coders would offset every letter by a single value (say 3).  But given today's tech, we can define even more advanced scrambling.  For example, if each letter is offset by a repeating pattern of different values, we can have values that represent different letters and letters represented by different values depending on the position. 

In [22]:
def letter_scramble(encoded_message):
    cypher = [1, 2, 3, 4]
    message_counter = 0
    cypher_counter = 0
    for _ in range(len(encoded_message)): #iterate based on length of the message
        if cypher_counter == 4: #reset counter length (% may replace this)
            cypher_counter = 0
        encoded_message[message_counter] += cypher[cypher_counter] #offset char at position given by counter by current cypher value   
        cypher_counter += 1 #increment counters
        message_counter += 1
    return encoded_message
message = Convert_Text("Where's the delivery man, I am hungry.")
print(letter_scramble(message))     

[88, 106, 104, 118, 102, 41, 118, 36, 117, 106, 104, 36, 101, 103, 111, 109, 119, 103, 117, 125, 33, 111, 100, 114, 45, 34, 76, 36, 98, 111, 35, 108, 118, 112, 106, 118, 122, 48]


Letter_scramble applies a Gronsfeld style cipher by changing each charachter by a different but systematic value.  

In [23]:
def letter_descramble(encoded_message):
    cypher = [-1, -2, -3, -4,]
    message_counter = 0
    cypher_counter = 0
    for _ in range(len(encoded_message)): #iterate through message
        if cypher_counter == 4: #reset counter length (code later uses % to replace)
            cypher_counter = 0
        encoded_message[message_counter] += cypher[cypher_counter] #undo offset using cipher key
        cypher_counter += 1
        message_counter += 1
    return encoded_message
message = [88, 106, 104, 118, 102, 41, 118, 36, 117, 106, 104, 36, 101, 103, 111, 109, 119, 103, 117, 125, 33, 111, 100, 114, 45, 34, 76, 36, 98, 111, 35, 108, 118, 112, 106, 118, 122, 48]
print(Convert_Num(letter_descramble(message)))
    

Where's the delivery man, I am hungry.


Letter_descramble reverses the cypher by subtracting the values that were added in the ecryption step.

In [24]:
def Encode_Scramble(n, e, message):
    m = letter_scramble(Convert_Text(message)) #Converts text into an integer list
    cipher_text = []
    for num in m: #iterate through our integer lest
        c = FME(num, Convert_Binary_String(e), n) # use FME to find (converted integer value)^e mod n. As defined, FME relies on e being converted to binary rep
                                                  # answer is an integer which represents our encrypted letter
        cipher_text.append(c) #populate list with our encrpted letters
    return cipher_text

print(letter_scramble(Convert_Text("Hi my name is bob")))
print(Convert_Text("Hi my name is bob"))

[73, 107, 35, 113, 122, 34, 113, 101, 110, 103, 35, 109, 116, 34, 101, 115, 99]
[72, 105, 32, 109, 121, 32, 110, 97, 109, 101, 32, 105, 115, 32, 98, 111, 98]


Encode_Scramble applies the letter_scramble function prior to passing values into the RSA algo.  Thus, if someone were to break n and find the private key, they would still not uncover the message.  The code below applies the Encode_Scramble function and shows the result and then applies only the regular Decode function to show how the message would appear by using the private key but no unscrambling.  

In [25]:
message = "It's a good day to send messages"
print(Encode_Scramble(2201, 1103, message))
print(Decode(2201, 1367,Encode_Scramble(2201, 1103, message)))

[611, 1524, 2120, 1308, 1806, 1049, 1769, 1568, 536, 1197, 758, 645, 884, 1049, 248, 645, 1912, 1197, 1769, 1308, 834, 536, 758, 645, 1656, 758, 1524, 1308, 1265, 239, 1593, 1308]
Jv*w!c#kpqg$ec|$uq#wfpg$ngvwbihw


In [26]:
def Decode_Scramble(n, d, cipher_text):
    message = '' 
    message_list = []
    for num in cipher_text: #iterate through our cipher text
        m = FME(num, Convert_Binary_String(d), n) # Use FME to find (encrypted digity)^d mod n
        message_list.append(m) # Append answer to list
    message = Convert_Num(letter_descramble(message_list)) #unsramble letters and convert back to a string
    return message

message = [611, 1524, 2120, 1308, 1806, 1049, 1769, 1568, 536, 1197, 758, 645, 884, 1049, 248, 645, 1912, 1197, 1769, 1308, 834, 536, 758, 645, 1656, 758, 1524, 1308, 1265, 239, 1593, 1308]


print(Decode_Scramble(2201, 1367, message))

It's a good day to send messages


Decode_Scramble incorperates the unscramble function and is shown below

In [27]:
message = "It's a good day to send messages"
print(Encode_Scramble(2201, 1103, message))
print(Decode_Scramble(2201, 1367,Encode_Scramble(2201, 1103, message)))

[611, 1524, 2120, 1308, 1806, 1049, 1769, 1568, 536, 1197, 758, 645, 884, 1049, 248, 645, 1912, 1197, 1769, 1308, 834, 536, 758, 645, 1656, 758, 1524, 1308, 1265, 239, 1593, 1308]
It's a good day to send messages


However, although the gronsfield cipher would be a good tool to scramble letters for a naive code breaker, it is still vulnerable to a technique called frequency analysis.  This technique looks to match patterns of encrypted letters to patterns of common english monograms, bigrams, and trigrams.  Using python, it is possible to quickly check all possible cipher codes and cross check this against this against common letter patterns.  For this demo and simplicity's sake, I will assume a cipher length of 3 and that space charchters are not encoded (only words).  I will also only focus on the frequency analysis and will assume that the RSA cipher has been decrypted.  I will also assume the coding is the same for upper and lower case letters.

In [28]:
def letter_scramble_Gronsfeld(_string, cipher):
    words = _string.lower().split() #convert string to lower case word list
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    counter = 0
    scrambled_word = ''
    scrambled_words = []
    for word in words: # iterate through words
        for letter in word: # iterate through letters of word
            if letter in alphabet: # prevents the code from attempting to convert non letter chars that are not in our dictionary
                new_index = (alphabet.index(letter) + cipher[counter]) % 26 #find letter's index in the alphabet, add current cipher
                scrambled_word += (alphabet[new_index]) #add scrambled letter to word
                counter += 1
                counter = counter % 3 #cipher is length 3
        scrambled_words.append(scrambled_word) #once all letters are iterated through, add scambled word to output list
        scrambled_word = '' # reset scrambled word
    return scrambled_words
cipher = [1,4,6]
print(letter_scramble_Gronsfeld("Yet here, Laertes! aboard, aboard, for shame! The wind sits in the shoulder of your sail, And you are stay’d for. There; my blessing with thee! And these few precepts in thy memory See thou character. Give thy thoughts no tongue, Nor any unproportioned thought his act. Be thou familiar, but by no means vulgar. Those friends thou hast, and their adoption tried, Grapple them to thy soul with hoops of steel; But do not dull thy palm with entertainment Of each new-hatch’d, unfledged comrade. Beware", cipher))


['ziz', 'iixf', 'pgfvzfw', 'gcsgsh', 'gcsgsh', 'lpv', 'yiesf', 'xnf', 'aooh', 'yjxy', 'jr', 'zii', 'yisamhks', 'sl', 'zsas', 'wgjp', 'goh', 'epy', 'gsi', 'yueee', 'jus', 'xnfvk', 'nc', 'hmiytmth', 'aoul', 'ziik', 'brj', 'ulkti', 'lfa', 'vsiiftzt', 'mt', 'ule', 'nispve', 'tik', 'uluv', 'gnbvgdxks', 'kowi', 'zic', 'zisahlzt', 'ru', 'usthyk', 'osx', 'bre', 'vrvssvpvzjstfh', 'zisahlz', 'imy', 'bgz', 'ci', 'zisa', 'gesjpobv', 'hvx', 'hz', 'ru', 'nigow', 'bvpmbv', 'zisyf', 'jxjitew', 'zisa', 'ieyu', 'ete', 'xnfmx', 'bhuqxopr', 'zsmke', 'kxbtvmi', 'ziis', 'us', 'zic', 'ypyr', 'xmzi', 'lupty', 'pj', 'yuikm', 'fau', 'hu', 'osz', 'eyrm', 'xnz', 'tgmq', 'cjxn', 'frzfvzbmtnitu', 'sl', 'feii', 'rkxlgugne', 'ytgpkekke', 'gunvgei', 'hfagsi']


One way to attack the gronsfield cipher is to iterate through possible ciphers and check the results against common trigrams.  The function below will be a helper funciton for the main Gronsfeld crack function.  This will check if the constructed trigram is a common english trigram.  Trigrams were taken from http://practicalcryptography.com/cryptanalysis/letter-frequencies-various-languages/english-letter-frequencies/ and transcribed into a list.

In [29]:
def is_trigram(_string): #takes in 3 letter string, checks if it is a common trigram
    trigrams = ["the", "and", "tha", "ent", "ing", "ion", "tio", "for", "nde", "has", "nce", "edt", "tis", "oft", "sth", "men", "her", "nth", "int", "ere", "ter", "est", "ers", "ati", "hat", "ate" , "all", " eth", "hes", "ver", "his", "oft", "ith", "fth", "sth", "oth", "res", "ont" ]
    for word in trigrams: 
        if _string == word:
            return 1 # return 1 if trigram found, 0 elsewise
    return 0
print(is_trigram("pop"))
print(is_trigram("the"))

0
1


In [30]:
def crack_cipher(encoded_message): # take in encoded message list, return possible ciphers
    cipher1 =[-1, -2, -3, -4, -5, -6, -7, -8, -9]
    cipher2 =[-1, -2, -3, -4, -5, -6, -7, -8, -9]
    cipher3 =[-1, -2, -3, -4, -5, -6, -7, -8, -9]
    alphabet = 'abcdefghijklmnopqrstuvwxyz'
    counter = 0
    score = 0
    max_score1 = 0
    max_score2 = 0
    code1 = ()
    code2 = ()
   
    for c1 in cipher1: 
        for c2 in cipher2: 
            for c3 in cipher3:
                for word in encoded_message: # iterate through encoded words, cipher1,cipher2, and cipher3
                    for _ in range(len(word) - 3): #we check 3 letters at a time, so when we stop 3 before the end of the word
                        if len(word) > 2: #bigrams and monograms not included in analysis for simplicity 
                            index_1 = (alphabet.index(word[counter]) +c1) % 26 #Check index of current letter, add c1 for new index
                            index_2 = (alphabet.index(word[((counter + 1) %26)]) + c2) % 26 #counter offset by 1 for letter in second position
                            index_3 = (alphabet.index(word[((counter + 2) %26)]) + c3) % 26 #mod 26 to wrap around the alphabet
                            new_word = alphabet[index_1]+alphabet[index_2] + alphabet[index_3] #construct new trigram using calulated indexes
                            score += is_trigram(new_word) #if trigram is a common trigram, score is incremented by 1, 0 elsewise
                            counter += 1 
                            counter = 0
                if score > max_score1: #after iterating through the word and calulating a score, check if this is max score
                    code2 = code1 #save second best score
                    max_score1,code1 = score, (c1,c2,c3) #c1,c2,c3 values for max score are saved 
                
                score = 0 
    return code1, code2
message = ['ziz', 'iixf', 'pgfvzfw', 'gcsgsh', 'gcsgsh', 'lpv', 'yiesf', 'xnf', 'aooh', 'yjxy', 'jr', 'zii', 'yisamhks', 'sl', 'zsas', 'wgjp', 'goh', 'epy', 'gsi', 'yueee', 'jus', 'xnfvk', 'nc', 'hmiytmth', 'aoul', 'ziik', 'brj', 'ulkti', 'lfa', 'vsiiftzt', 'mt', 'ule', 'nispve', 'tik', 'uluv', 'gnbvgdxks', 'kowi', 'zic', 'zisahlzt', 'ru', 'usthyk', 'osx', 'bre', 'vrvssvpvzjstfh', 'zisahlz', 'imy', 'bgz', 'ci', 'zisa', 'gesjpobv', 'hvx', 'hz', 'ru', 'nigow', 'bvpmbv', 'zisyf', 'jxjitew', 'zisa', 'ieyu', 'ete', 'xnfmx', 'bhuqxopr', 'zsmke', 'kxbtvmi', 'ziis', 'us', 'zic', 'ypyr', 'xmzi', 'lupty', 'pj', 'yuikm', 'fau', 'hu', 'osz', 'eyrm', 'xnz', 'tgmq', 'cjxn', 'frzfvzbmtnitu', 'sl', 'feii', 'rkxlgugne', 'ytgpkekke', 'gunvgei', 'hfagsi']
print(crack_cipher(message))

((-6, -4, -1), (-1, -4, -6))


The code below demonstrates a shakespear passage encrypted with the gronsfeld cipher and then the attack on the cipher using the code above.  In this example, the program successfully returns the shift that needs to be applied to reverse the encyption.  The code also provides the second best option to reverse the cypher.

In [31]:
cipher = (1,4,6)
message =  "Yet here, Laertes! aboard, aboard, for shame! The wind sits in the shoulder of your sail, And you are stay’d for. There; my blessing with thee! And these few precepts in thy memory See thou character. Give thy thoughts no tongue, Nor any unproportioned thought his act. Be thou familiar, but by no means vulgar. Those friends thou hast, and their adoption tried, Grapple them to thy soul with hoops of steel; But do not dull thy palm with entertainment Of each new-hatch’d, unfledged comrade. Beware"
print(letter_scramble_Gronsfeld(message, cipher))
print(crack_cipher(letter_scramble_Gronsfeld(message, cipher)))


['ziz', 'iixf', 'pgfvzfw', 'gcsgsh', 'gcsgsh', 'lpv', 'yiesf', 'xnf', 'aooh', 'yjxy', 'jr', 'zii', 'yisamhks', 'sl', 'zsas', 'wgjp', 'goh', 'epy', 'gsi', 'yueee', 'jus', 'xnfvk', 'nc', 'hmiytmth', 'aoul', 'ziik', 'brj', 'ulkti', 'lfa', 'vsiiftzt', 'mt', 'ule', 'nispve', 'tik', 'uluv', 'gnbvgdxks', 'kowi', 'zic', 'zisahlzt', 'ru', 'usthyk', 'osx', 'bre', 'vrvssvpvzjstfh', 'zisahlz', 'imy', 'bgz', 'ci', 'zisa', 'gesjpobv', 'hvx', 'hz', 'ru', 'nigow', 'bvpmbv', 'zisyf', 'jxjitew', 'zisa', 'ieyu', 'ete', 'xnfmx', 'bhuqxopr', 'zsmke', 'kxbtvmi', 'ziis', 'us', 'zic', 'ypyr', 'xmzi', 'lupty', 'pj', 'yuikm', 'fau', 'hu', 'osz', 'eyrm', 'xnz', 'tgmq', 'cjxn', 'frzfvzbmtnitu', 'sl', 'feii', 'rkxlgugne', 'ytgpkekke', 'gunvgei', 'hfagsi']
((-6, -4, -1), (-1, -4, -6))


The code below demonstratets that the attack on the gronsfeld cipher is not necessarily perfect.  Another shakespearean passsage is encrypted, however the correct cypher is not returned.  Interestingly, the code does return the the values of the shifts, but not in the correct position.

In [32]:
cipher = [9, 2, 3]
message = "We know all men are not created equal in the sense some people would have us believe—some people are smarter than others, some people have more opportunity because they’re born with it, some men make more money than others, some ladies make better cakes than others—some people are born gifted beyond the normal scope of most men."
print(crack_cipher(letter_scramble_Gronsfeld(message, cipher)))

((-3, -9, -2), (-3, -6, -1))


Finally, a monologue from the movie Good Will Hunting is encrypted below and successfully attacked.

In [33]:
cipher = (1, 4, 3)
message = "If I asked you about art you’d probably give me the skinny on every art book ever written. Michelangelo? You know a lot about him. Life’s work, political aspirations, him and the pope, sexual orientation, the whole works, right? But I bet you can’t tell me what it smells like in the Sistine Chapel. You’ve never actually stood there and looked up at that beautiful ceiling. Seen that. If I asked you about women you’d probably give me a syllabus of your personal favourites. You may have even been laid a few times. But you can’t tell me what it feels like to wake up next to a woman and feel truly happy. You’re a tough kid."
print(crack_cipher(letter_scramble_Gronsfeld(message, cipher)))

((-2, -6, -2), (-1, -2, -4))


The success of the attack likely depends on how often the passage being decrypted uses the trigrams that the attack checks for.  The attack could be made stronger with additional tweaks.  First, it could check additional trigrams and also compare trigram frequency in the passage to the expected trigram frequency frequently seen in english.  Second, scores for bigram and letter frequency could also be incorperated for a more robust analysis of the text.  The gronsfeld cipher is a a modified version of the Vigenere Cipher in which each letter is offset by a value given by a keyword.  The Vigenere Cipher is expanded as each letter can be mapped to 26 different letters where as traditionally, the gronsfeld cipher limits the shit based on digits 1-9.  However, even the expanded vigenere cipher is vulnerable to the same sort of iterative freqency analysis.  Thus, more advanced techniques for letter scrambling are required to prevent against attack by frequency analysis.    